In [1]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import matplotlib.dates as mdates
from matplotlib.ticker import MaxNLocator

import aplpy
from skimage.feature import peak_local_max
from scipy.stats import chi2, norm

from astropy.wcs import WCS
from astropy.io import fits
from astropy import units as u
from astropy.coordinates import SkyCoord, Angle, search_around_sky
from astropy.wcs.utils import skycoord_to_pixel, pixel_to_skycoord
from astropy.nddata.utils import Cutout2D
from astropy.table import Table, vstack
from astropy.time import Time

In [2]:
import glob
import os

In [3]:
src_name = "J074249.01-282244.23"
sbid = 50660
beam = 28
folder = f"/import/eve/fast_imaging/SB{sbid}/images"

In [6]:
imagelist = []
for size in ["?", "??", "???", "????"]:
    tmp = glob.glob(os.path.join(folder, f'*{beam}_{size}.fits'))
    tmp.sort()
    imagelist += tmp

# print(imagelist)

In [8]:
ind = 55
image = imagelist[ind]
print(image)

/import/eve/fast_imaging/SB50660/images/SB50660_beam28_57.fits


In [9]:
src = SkyCoord(src_name, unit=(u.hourangle, u.degree))
src

<SkyCoord (ICRS): (ra, dec) in deg
    (115.70420833, -28.37895278)>

In [10]:
def save_fits_cutout(src_name, image, radius=5):
    
    src = SkyCoord(src_name, unit=(u.hourangle, u.degree))
    # open the fits
    hdu = fits.open(image)[0]
    # get the fits data (drop-out extra dimensions)
    data = hdu.data.squeeze()
    # get wcs frame
    wcs = WCS(header=hdu.header, naxis=2)
    print(wcs)
    # generate cutout
    cutout = Cutout2D(data, 
                      position=src, 
                      size=radius*u.arcmin, 
                      wcs=wcs
                     )

    # Put the cutout image in the FITS HDU
    hdu.data = cutout.data

    # Update the FITS header with the cutout WCS
    hdu.header.update(cutout.wcs.to_header())

    # Write the cutout to a new FITS file
    cutout_filename = 'example_cutout.fits'
    hdu.writeto(cutout_filename, overwrite=True)

    return cutout

In [28]:
def save_fits_cube(src_name, imagelist, radius=5):
    """Generate a fits cube containing images cutout in given position. 
    src: 
        astropy Skycoord object. 
    imagelist: 
        A list of short FITS images, in correct order. 
    radius:
        unit of arcmin, the cutout size
    """
    
    # get the source position 
    src = SkyCoord(src_name, unit=(u.hourangle, u.degree))
    print("Get source position ({}, {})...".format(src.ra.degree, src.dec.degree))

    final_data = []
    
    # generate each frame
    for i, image in enumerate(imagelist):
        print("Processing image number %s/%s" % (i, len(imagelist)))
        
        # open the fits
        hdu = fits.open(image)[0]
        # get the fits data (drop-out extra dimensions)
        data = hdu.data.squeeze()
        # get wcs frame
        wcs = WCS(header=hdu.header, naxis=2)
        # generate cutout
        cutout = Cutout2D(data, 
                          position=src, 
                          size=radius*u.arcmin, 
                          wcs=wcs)
        # Put the cutout image in the FITS HDU
        final_data.append(cutout.data)

        if i == 0:
            final_header = cutout.wcs.to_header()
            final_hdu = hdu
    
    # Update the FITS header with the cutout WCS
    final_hdu.header.update(final_header)

    # save the data cube to FITS
    final_hdu.data = np.array(final_data)

    # Write the cutout to a new FITS file
    cutout_filename = 'example_cube.fits'
    final_hdu.writeto(cutout_filename, overwrite=True)

    return final_hdu
    

In [11]:
cutout = save_fits_cutout(src, image)

WCS Keywords

Number of WCS axes: 2
CTYPE : 'RA---SIN'  'DEC--SIN'  
CRVAL : 116.0649258267  -27.75440551077  
CRPIX : 1401.0  1401.0  
PC1_1 PC1_2  : 1.0  0.0  
PC2_1 PC2_2  : 0.0  1.0  
CDELT : -0.0006944444444444  0.0006944444444444  
NAXIS : 2800  2800  1  1


Set OBSGEO-B to   -26.704100 from OBSGEO-[XYZ].
Set OBSGEO-H to      121.995 from OBSGEO-[XYZ]'. [astropy.wcs.wcs]


In [12]:
cutout.data.shape

(120, 120)

In [13]:
cutout.wcs

WCS Keywords

Number of WCS axes: 2
CTYPE : 'RA---SIN'  'DEC--SIN'  
CRVAL : 116.0649258267  -27.75440551077  
CRPIX : -396.0  961.0  
PC1_1 PC1_2  : 1.0  0.0  
PC2_1 PC2_2  : 0.0  1.0  
CDELT : -0.0006944444444444  0.0006944444444444  
NAXIS : 120  120

In [29]:
hdu = save_fits_cube(src_name, imagelist)

Get source position (115.70420833333333, -28.37895277777778)...
Processing image number 0/69
Processing image number 1/69
Processing image number 2/69
Processing image number 3/69
Processing image number 4/69
Processing image number 5/69
Processing image number 6/69


Set OBSGEO-B to   -26.704100 from OBSGEO-[XYZ].
Set OBSGEO-H to      121.995 from OBSGEO-[XYZ]'. [astropy.wcs.wcs]
Set OBSGEO-B to   -26.704100 from OBSGEO-[XYZ].
Set OBSGEO-H to      121.995 from OBSGEO-[XYZ]'. [astropy.wcs.wcs]
Set OBSGEO-B to   -26.704100 from OBSGEO-[XYZ].
Set OBSGEO-H to      121.995 from OBSGEO-[XYZ]'. [astropy.wcs.wcs]
Set OBSGEO-B to   -26.704100 from OBSGEO-[XYZ].
Set OBSGEO-H to      121.995 from OBSGEO-[XYZ]'. [astropy.wcs.wcs]
Set OBSGEO-B to   -26.704100 from OBSGEO-[XYZ].
Set OBSGEO-H to      121.995 from OBSGEO-[XYZ]'. [astropy.wcs.wcs]
Set OBSGEO-B to   -26.704100 from OBSGEO-[XYZ].
Set OBSGEO-H to      121.995 from OBSGEO-[XYZ]'. [astropy.wcs.wcs]
Set OBSGEO-B to   -26.704100 from OBSGEO-[XYZ].
Set OBSGEO-H to      121.995 from OBSGEO-[XYZ]'. [astropy.wcs.wcs]
Set OBSGEO-B to   -26.704100 from OBSGEO-[XYZ].
Set OBSGEO-H to      121.995 from OBSGEO-[XYZ]'. [astropy.wcs.wcs]
Set OBSGEO-B to   -26.704100 from OBSGEO-[XYZ].
Set OBSGEO-H to      121.995 fro

Processing image number 7/69
Processing image number 8/69
Processing image number 9/69
Processing image number 10/69
Processing image number 11/69
Processing image number 12/69
Processing image number 13/69
Processing image number 14/69
Processing image number 15/69


Set OBSGEO-B to   -26.704100 from OBSGEO-[XYZ].
Set OBSGEO-H to      121.995 from OBSGEO-[XYZ]'. [astropy.wcs.wcs]
Set OBSGEO-B to   -26.704100 from OBSGEO-[XYZ].
Set OBSGEO-H to      121.995 from OBSGEO-[XYZ]'. [astropy.wcs.wcs]
Set OBSGEO-B to   -26.704100 from OBSGEO-[XYZ].
Set OBSGEO-H to      121.995 from OBSGEO-[XYZ]'. [astropy.wcs.wcs]
Set OBSGEO-B to   -26.704100 from OBSGEO-[XYZ].
Set OBSGEO-H to      121.995 from OBSGEO-[XYZ]'. [astropy.wcs.wcs]
Set OBSGEO-B to   -26.704100 from OBSGEO-[XYZ].
Set OBSGEO-H to      121.995 from OBSGEO-[XYZ]'. [astropy.wcs.wcs]
Set OBSGEO-B to   -26.704100 from OBSGEO-[XYZ].
Set OBSGEO-H to      121.995 from OBSGEO-[XYZ]'. [astropy.wcs.wcs]
Set OBSGEO-B to   -26.704100 from OBSGEO-[XYZ].
Set OBSGEO-H to      121.995 from OBSGEO-[XYZ]'. [astropy.wcs.wcs]
Set OBSGEO-B to   -26.704100 from OBSGEO-[XYZ].
Set OBSGEO-H to      121.995 from OBSGEO-[XYZ]'. [astropy.wcs.wcs]
Set OBSGEO-B to   -26.704100 from OBSGEO-[XYZ].
Set OBSGEO-H to      121.995 fro

Processing image number 16/69
Processing image number 17/69
Processing image number 18/69
Processing image number 19/69
Processing image number 20/69
Processing image number 21/69
Processing image number 22/69
Processing image number 23/69
Processing image number 24/69


...

Set OBSGEO-B to   -26.704100 from OBSGEO-[XYZ].
Set OBSGEO-H to      121.995 from OBSGEO-[XYZ]'. [astropy.wcs.wcs]
Set OBSGEO-B to   -26.704100 from OBSGEO-[XYZ].
Set OBSGEO-H to      121.995 from OBSGEO-[XYZ]'. [astropy.wcs.wcs]
Set OBSGEO-B to   -26.704100 from OBSGEO-[XYZ].
Set OBSGEO-H to      121.995 from OBSGEO-[XYZ]'. [astropy.wcs.wcs]
Set OBSGEO-B to   -26.704100 from OBSGEO-[XYZ].
Set OBSGEO-H to      121.995 from OBSGEO-[XYZ]'. [astropy.wcs.wcs]
Set OBSGEO-B to   -26.704100 from OBSGEO-[XYZ].
Set OBSGEO-H to      121.995 from OBSGEO-[XYZ]'. [astropy.wcs.wcs]
Set OBSGEO-B to   -26.704100 from OBSGEO-[XYZ].
Set OBSGEO-H to      121.995 from OBSGEO-[XYZ]'. [astropy.wcs.wcs]
Set OBSGEO-B to   -26.704100 from OBSGEO-[XYZ].
Set OBSGEO-H to      121.995 from OBSGEO-[XYZ]'. [astropy.wcs.wcs]
Set OBSGEO-B to   -26.704100 from OBSGEO-[XYZ].
Set OBSGEO-H to      121.995 from OBSGEO-[XYZ]'. [astropy.wcs.wcs]
Set OBSGEO-B to   -26.704100 from OBSGEO-[XYZ].
Set OBSGEO-H to      121.995 fro

Processing image number 46/69
Processing image number 47/69
Processing image number 48/69
Processing image number 49/69
Processing image number 50/69
Processing image number 51/69
Processing image number 52/69
Processing image number 53/69
Processing image number 54/69
Processing image number 55/69
Processing image number 56/69


Set OBSGEO-B to   -26.704100 from OBSGEO-[XYZ].
Set OBSGEO-H to      121.995 from OBSGEO-[XYZ]'. [astropy.wcs.wcs]
Set OBSGEO-B to   -26.704100 from OBSGEO-[XYZ].
Set OBSGEO-H to      121.995 from OBSGEO-[XYZ]'. [astropy.wcs.wcs]
Set OBSGEO-B to   -26.704100 from OBSGEO-[XYZ].
Set OBSGEO-H to      121.995 from OBSGEO-[XYZ]'. [astropy.wcs.wcs]
Set OBSGEO-B to   -26.704100 from OBSGEO-[XYZ].
Set OBSGEO-H to      121.995 from OBSGEO-[XYZ]'. [astropy.wcs.wcs]
Set OBSGEO-B to   -26.704100 from OBSGEO-[XYZ].
Set OBSGEO-H to      121.995 from OBSGEO-[XYZ]'. [astropy.wcs.wcs]
Set OBSGEO-B to   -26.704100 from OBSGEO-[XYZ].
Set OBSGEO-H to      121.995 from OBSGEO-[XYZ]'. [astropy.wcs.wcs]
Set OBSGEO-B to   -26.704100 from OBSGEO-[XYZ].
Set OBSGEO-H to      121.995 from OBSGEO-[XYZ]'. [astropy.wcs.wcs]
Set OBSGEO-B to   -26.704100 from OBSGEO-[XYZ].
Set OBSGEO-H to      121.995 from OBSGEO-[XYZ]'. [astropy.wcs.wcs]
Set OBSGEO-B to   -26.704100 from OBSGEO-[XYZ].
Set OBSGEO-H to      121.995 fro

Processing image number 57/69
Processing image number 58/69
Processing image number 59/69
Processing image number 60/69
Processing image number 61/69
Processing image number 62/69
Processing image number 63/69
Processing image number 64/69
Processing image number 65/69
Processing image number 66/69
Processing image number 67/69
Processing image number 68/69


Set OBSGEO-B to   -26.704100 from OBSGEO-[XYZ].
Set OBSGEO-H to      121.995 from OBSGEO-[XYZ]'. [astropy.wcs.wcs]
Set OBSGEO-B to   -26.704100 from OBSGEO-[XYZ].
Set OBSGEO-H to      121.995 from OBSGEO-[XYZ]'. [astropy.wcs.wcs]
Set OBSGEO-B to   -26.704100 from OBSGEO-[XYZ].
Set OBSGEO-H to      121.995 from OBSGEO-[XYZ]'. [astropy.wcs.wcs]
Set OBSGEO-B to   -26.704100 from OBSGEO-[XYZ].
Set OBSGEO-H to      121.995 from OBSGEO-[XYZ]'. [astropy.wcs.wcs]
Set OBSGEO-B to   -26.704100 from OBSGEO-[XYZ].
Set OBSGEO-H to      121.995 from OBSGEO-[XYZ]'. [astropy.wcs.wcs]
Set OBSGEO-B to   -26.704100 from OBSGEO-[XYZ].
Set OBSGEO-H to      121.995 from OBSGEO-[XYZ]'. [astropy.wcs.wcs]
Set OBSGEO-B to   -26.704100 from OBSGEO-[XYZ].
Set OBSGEO-H to      121.995 from OBSGEO-[XYZ]'. [astropy.wcs.wcs]


In [30]:
hdu.header

SIMPLE  =                    T /Standard FITS                                   
BITPIX  =                  -32 /Floating point (32 bit)                         
NAXIS   =                    3                                                  
NAXIS1  =                  120                                                  
NAXIS2  =                  120                                                  
NAXIS3  =                   69                                                  
EXTEND  =                    T                                                  
BSCALE  =   1.000000000000E+00 /PHYSICAL = PIXEL*BSCALE + BZERO                 
BZERO   =   0.000000000000E+00                                                  
BMAJ    =   4.568852318658E-03                                                  
BMIN    =   4.058241844177E-03                                                  
BPA     =   8.230576324463E+01                                                  
BTYPE   = 'Intensity'       